# Imports

In [52]:
#misc
import logging
import os
import pprint
logging.basicConfig(level=logging.INFO)

import multiprocessing
print("cores", multiprocessing.cpu_count())
from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

# http
import requests
import json
from datetime import datetime, date, timedelta
import time
from bs4 import BeautifulSoup


# data viz
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

# data
import numpy as np
#import dask.dataframe as pd
import pandas as pd
import pickle
from collections import defaultdict
import re
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)
from copy import deepcopy
import random

#nlp
import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
from sense2vec import Sense2VecComponent
from spacy import displacy
from tqdm import tqdm_notebook, notebook

import gensim
import gensim.corpora as corpora
from gensim import matutils
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel
from gensim.models.phrases import Phrases, Phraser

import pyLDAvis.gensim
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation

import nltk
from nltk import tokenize
#nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stop_words = stopwords.words('english')
# Constants
RANDOM_STATE=42
SAMPLE_SIZE=2000
WORKERS=3

cores 8


# Read saved sample

In [ ]:
#df_sample = df_full.head(20000).sample(SAMPLE_SIZE, random_state=RANDOM_STATE)
df_year_sample = pd.read_pickle('./reddit_df_19_sub.pkl')

In [2]:
df_sample = pd.read_pickle('./reddit_df_clean_sub.pkl')

In [3]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3200 entries, 198110 to 222945
Data columns (total 9 columns):
author           3200 non-null object
created_utc      3200 non-null datetime64[ns]
created_year     3200 non-null int64
created_month    3200 non-null int64
score            3200 non-null int64
num_comments     3200 non-null int64
full_link        3200 non-null object
title            3200 non-null object
selftext         3200 non-null object
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 250.0+ KB


In [22]:
df_sample[df_sample['selftext'].str.contains('ragna')]

author         created_utc  created_year  created_month  \
112244  so8so            2015-11-19 01:32:50  2015          11              
13776   Kiui-Pai         2012-04-01 18:54:18  2012          4               
43220   mollywog94       2013-08-10 22:09:42  2013          8               
59587   newhomeownr      2014-05-29 05:09:59  2014          5               
271367  OffsetWishes     2019-09-22 18:30:11  2019          9               
195888  ImJustC0nfused   2018-01-13 18:48:27  2018          1               
143667  areyouscaredyet_ 2016-09-11 22:30:30  2016          9               
184100  to_Bewilder      2017-09-05 22:46:01  2017          9               
24263   Forever_King     2012-09-16 07:52:30  2012          9               
17331   [deleted]        2012-06-08 16:43:09  2012          6               
253641  Nickbotic        2019-05-07 11:00:03  2019          5               

        score  num_comments  \
112244  664    181            
13776   177    25             
43220   181    76             
59587   216    37             
271367  207    18             
195888  1594   327            
143667  335    51             
184100  120    5              
24263   1021   275            
17331   160    25             
253641  3915   260            

                                                                                                  full_link  \
112244  https://www.reddit.com/r/nosleep/comments/3tdgmb/i_found_a_locked_room_in_the_basement_of_my_new/     
13776   https://www.reddit.com/r/nosleep/comments/roam3/where_are_you/                                        
43220   https://www.reddit.com/r/nosleep/comments/1k40v1/texts_from_my_boyfriend/                             
59587   https://www.reddit.com/r/nosleep/comments/26rhij/i_came_home_today_and_someone_else_has_been_in_my/   
271367  https://www.reddit.com/r/nosleep/comments/d7uf6a/our_town_was_been_given_the_warning_for_a_major/     
195888  https://www.reddit.com/r/nosleep/comments/7q6bgt/i_ran_into_my_high_school_sweetheart_tonight_at/     
143667  https://www.reddit.com/r/nosleep/comments/52b3mt/are_your_doors_locked/                               
184100  https://www.reddit.com/r/nosleep/comments/6ybl53/i_was_a_house_sitter/                                
24263   https://www.reddit.com/r/nosleep/comments/zypxi/8th_grade_math_3final/                                
17331   https://www.reddit.com/r/nosleep/comments/ursc0/the_answering_machine/                                
253641  https://www.reddit.com/r/nosleep/comments/blpau1/i_just_found_the_letters_my_best_friend_and_i/       

                                                                                                             title  \
112244  I found a locked room in the basement of my new house. Opening it was a really big mistake. [Update 1]       
13776   Where are you?                                                                                               
43220   Texts from my boyfriend.                                                                                     
59587   I came home today, and someone else has been in my house messing with things..                               
271367  Our town was been given the warning for a major storm, but there’s something worse.                          
195888  I ran into my high school sweetheart tonight at my 10-year reunion… the one I married? (Part 2)              
143667  Are your doors locked?                                                                                       
184100  I was a House Sitter                                                                                         
24263   8th Grade Math (3)(Final)                                                                                    
17331   The Answering Machine                                                                                        
253641  I just found the letters my best friend and I used to send each other when he mov

# Preprocess

## Stopwords

In [59]:
stop_words_pd = pd.read_csv('news-stopwords/sw1k.csv')
stop_words_pd.dropna(subset=['term'], inplace=True)
nltk_stop_words.extend(stop_words_pd['term'].tolist())

english_lit = 'faye, mel, morrison, natasha, solomon, josie, liz, tess, cameron, aiden, steph, luke, lucas, josh, matt, amelia, dylan, jake, pete, ethan, gabe, sandler, jessica, sander, dee, nate, tasha, trevor, candice, janie, cameron, willy, mikey, tess, alexs, avi, zack, lindsay, angie, avery, patty, chrissy, jude, finn, spencer, riley, kelsey, alyssa, meredith, savannah, evan, stan, sophie, timmy, jess, joey, liam, alex, nick, angela, alex, emma, lily, phil, sam, oliver, eric, claire, Mary, Katherine, Josephine, Ana, Heidi, Patricia, Joan, Thelma, Renee, Glenda, Linda, Ashley, Shannon, Ida, Lydia, Barbara, Judith, Sheila, Vivian, Viola, Elizabeth, Rose, Ethel, Roberta, Courtney, Jennifer, Janice, Ellen, Holly, Marian, Maria, Kelly, Elaine, Brittany, Stella, Susan, Nicole, Marjorie, Melanie, Caroline, Margaret, Judy, Carrie, Loretta, Dora, Dorothy, Christina, Charlotte, Yolanda, Jo, Lisa, Kathy, Monica, Jeanette, Vickie, Nancy, Theresa, Esther, Laurie, Mattie, Karen, Beverly, Pauline, Katie, Terry, Betty, Denise, Emma, Kristen, Maxine, Helen, Tammy, Juanita, Vanessa, Irma, Sandra, Irene, Anita, Alma, Mabel, Donna, Jane, Rhonda, Sue, Marsha, Carol, Lori, Hazel, Elsie, Myrtle, Ruth, Rachel, Amber, Beth, Lena, Sharon, Marilyn, Eva, Jeanne, Christy, Michelle, Andrea, Debbie, Vicki, Deanna, Laura, Kathryn, April, Carla, Patsy, Sarah, Louise, Leslie, Tara, Hilda, Kimberly, Sara, Clara, Rosemary, Gwendolyn, Deborah, Anne, Lucille, Eileen, Jennie, Jessica, Jacqueline, Jamie, Terri, Nora, Shirley, Wanda, Joanne, Gertrude, Margie, Cynthia, Bonnie, Eleanor, Lucy, Nina, Angela, Julia, Valerie, Tonya, Cassandra, Melissa, Ruby, Danielle, Ella, Leah, Brenda, Lois, Megan, Stacey, Penny, Amy, Tina, Alicia, Wilma, Kay, Anna, Phyllis, Suzanne, Gina, Priscilla, Rebecca, Norma, Michele, Kristin, Naomi, Virginia, Paula, Gail, Jessie, Carole, Kathleen, Diana, Bertha, Natalie, Brandy, Pamela, Annie, Darlene, Agnes, Olga, Martha, Lillian, Veronica, Vera, Billie, Debra, Emily, Jill, Willie, Dianne, Amanda, Robin, Erin, Charlene, Tracey, Stephanie, Peggy, Geraldine, Bessie, Leona, Carolyn, Crystal, Lauren, Delores, Jenny, Christine, Gladys, Cathy, Melinda, Felicia, Marie, Rita, Joann, Pearl, Sonia, Janet, Dawn, Lorraine, Arlene, Miriam, Catherine, Connie, Lynn, Maureen, Velma, Frances, Florence, Sally, Colleen, Becky, Ann, Tracy, Regina, Allison, Bobbie, Joyce, Edna, Erica, Tamara, Violet, Diane, Tiffany, Beatrice, Joy, Kristina, Alice, Carmen, Dolores, Georgia, Toni, Julie, Rosa, Bernice, Constance, Misty, Heather, Cindy, Audrey, Lillie, Mae, Teresa, Grace, Yvonne, Claudia, Shelly, Doris, Wendy, Annette, Jackie, Daisy, Gloria, Victoria, June, Marcia, Ramona, Evelyn, Edith, Samantha, Tanya, Sherri, Jean, Kim, Marion, Nellie, Erika, Cheryl, Sherry, Dana, Minnie, Katrina, Mildred, Sylvia, Stacy, Marlene, Claire , James, Willie, Chad, Zachary, Mathew, John, Ralph, Jacob, Corey, Tyrone, Robert, Lawrence, Lee, Herman, Darren, Michael, Nicholas, Melvin, Maurice, Lonnie, William, Roy, Alfred, Vernon, Lance, David, Benjamin, Kyle, Roberto, Cody, Richard, Bruce, Francis, Clyde, Julio, Charles, Brandon, Bradley, Glen, Kelly, Joseph, Adam, Jesus, Hector, Kurt, Thomas, Harry, Herbert, Shane, Allan, Christopher, Fred, Frederick, Ricardo, Nelson, Daniel, Wayne, Ray, Sam, Guy, Paul, Billy, Joel, Rick, Clayton, Mark, Steve, Edwin, Lester, Hugh, Donald, Louis, Don, Brent, Max, George, Jeremy, Eddie, Ramon, Dwayne, Kenneth, Aaron, Ricky, Charlie, Dwight, Steven, Randy, Troy, Tyler, Armando, Edward, Howard, Randall, Gilbert, Felix, Brian, Eugene, Barry, Gene, Jimmie, Ronald, Carlos, Alexander, Marc, Everett, Anthony, Russell, Bernard, Reginald, Jordan, Kevin, Bobby, Mario, Ruben, Ian, Jason, Victor, Leroy, Brett, Wallace, Matthew, Martin, Francisco, Angel, Ken, Gary, Ernest, Marcus, Nathaniel, Bob, Timothy, Phillip, Micheal, Rafael, Jaime, Jose, Todd, Theodore, Leslie, Casey, Larry, Jesse, Clifford, Edgar, Alfredo, Jeffrey, Craig, Miguel, Milton, Alberto, Frank, Alan, Oscar, Raul, Dave, Scott, Shawn, Jay, Ben, Ivan, Eric, Clarence, Jim, Chester, Johnnie, Stephen, Sean, Tom, Cecil, Sidney, Andrew, Philip, Calvin, Duane, Byron, Raymond, Chris, Alex, Franklin, Julian, Gregory, Johnny, Jon, Andre, Isaac, Joshua, Earl, Ronnie, Elmer, Morris, Jerry, Jimmy, Bill, Brad, Clifton, Dennis, Antonio, Lloyd, Gabriel, Willard, Walter, Danny, Tommy, Ron, Daryl, Patrick, Bryan, Leon, Mitchell, Ross, Peter, Tony, Derek, Roland, Virgil, Harold, Luis, Warren, Arnold, Andy, Douglas, Mike, Darrell, Harvey, Marshall, Henry, Stanley, Jerome, Jared, Salvador, Carl, Leonard, Floyd, Adrian, Perry, Arthur, Nathan, Leo, Karl, Kirk, Ryan, Dale, Alvin, Cory, Sergio, Roger, Manuel, Tim, Claude, Marion, Joe, Rodney, Wesley, Erik, Tracy, Juan, Curtis, Gordon, Darryl, Seth, Jack, Norman, Dean, Jamie, Kent, Albert, Allen, Greg, Neil, Terrance, Jonathan, Marvin, Jorge, Jessie, Rene, Justin, Vincent, Dustin, Christian, Eduardo, Terry, Glenn, Pedro, Javier, Terrence, Gerald, Jeffery, Derrick, Fernando, Enrique, Keith, Travis, Dan, Clinton, Freddie, Samuel, Jeff, Lewis, Ted, Wade'
english_lit_stopwords = english_lit.lower().split(', ')
nltk_stop_words.extend(english_lit_stopwords)

nltk_stop_words.extend([
    'know', 'like', 'look', 'nosleep', 'reddit', 'post', '’s', "\'s", "'s",
    'stuff', 'subreddit', 'gt', 'lt', 'en',  'r', 'comment', 'comments',
    'gon', 'na', 'shit', 'guys', 'yeah', 'damn', 'dude', 'guess', 'ass',
    'mods', 'update', 'part', 'one', 'two', 'three', 'four', 'five', 'six',
    'seven', 'eight', 'nine', 'nsfw', 'previous', 'events', 'information',
    'edit', 'edited', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', 
    'simply', 'anyway', 'whatever', 'fine', 'do', 'not', 's', 'r', 'kinda',
    'so', 'sometimes', 'maybe', 'ok'
])

stop_words = nltk_stop_words

## Pipeline

In [60]:
def sent_to_words(docs):
    for doc in docs:
        yield(gensim.utils.simple_preprocess(str(doc), deacc=True))  # deacc=True removes punctuations
        
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in tqdm_notebook(texts)]

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in tqdm_notebook(texts)]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in tqdm_notebook(texts):
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def process_data(data):
    # Remove Emails
    data = [re.sub(r'\S*@\S*\s?', '', doc) for doc in data]

    # Remove long chains of xxxs, these are pseudo redactions
    data = [re.sub(r'xxx?', '', doc) for doc in data]

    # Replace long chains of ooos
    data = [re.sub(r'oo+\b', 'o', doc) for doc in data]

    # Remove URLs
    data = [re.sub(r'http\S+', '', doc) for doc in data]

    # Remove new line characters
    data = [re.sub(r'\s+', ' ', doc) for doc in data]

    # Remove distracting single quotes
    data = [re.sub("\'", "", doc) for doc in data]

    # lowercase
    data = [doc.lower() for doc in data]

    data_words = list(sent_to_words(data))

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)

    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # keeping only tagger component (for efficiency)
    nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv, proper nount
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN'])

    # second pass on lemmatized words
    data_lemmatized = remove_stopwords(data_lemmatized)


    print(data_lemmatized[:1])
    return data_lemmatized


## Topic Modeling (3,600 docs of 250,000, 1.4%)

Acronynms: Latent Semantic Analysis (LSA) is just another name for 
Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)

In [61]:
data_docs = df_sample['selftext'].tolist()

In [62]:
data_lemmatized = process_data(data_docs)

INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:collected 1253232 word types from a corpus of 5426165 words (unigram + bigrams) and 3200 sentences
INFO:gensim.models.phrases:using 1253232 counts as vocab in Phrases<0 vocab, min_count=5, threshold=50, max_vocab_size=40000000>
INFO:gensim.models.phrases:source_vocab length 1253232
INFO:gensim.models.phrases:Phraser built with 2269 phrasegrams
//anaconda3/envs/reddit/lib/python3.7/site-packages/tqdm/__init__.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  TqdmDeprecationWarning)


//anaconda3/envs/reddit/lib/python3.7/site-packages/tqdm/__init__.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  TqdmDeprecationWarning)


//anaconda3/envs/reddit/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
//anaconda3/envs/reddit/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
//anaconda3/envs/reddit/lib/python3.7/site-packages/tqdm/__init__.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  TqdmDeprecationWarning)


//anaconda3/envs/reddit/lib/python3.7/site-packages/tqdm/__init__.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  TqdmDeprecationWarning)



[['maddie', 'confuse', 'learn', 'type', 'legalize', 'marriage', 'ms', 'jone', 'confuse', 'nice', 'brother', 'sister', 'aunt', 'wait', 'mommy', 'dad', 'sister', 'ms', 'jone', 'type', 'exist', 'dinner', 'mommy', 'dad', 'cool', 'nice', 'mommy', 'dad', 'lie', 'sister', 'dad', 'dark', 'fast', 'concerned', 'whisper', 'impaginery', 'sikiatrist', 'dinner', 'upset', 'upstairs', 'sister', 'mad', 'crying', 'forget', 'sad', 'cry', 'ms', 'jone', 'kid', 'sad', 'kid', 'ms', 'jone', 'funny', 'smile', 'sister', 'breathe', 'sister', 'friend', 'sister', 'funny', 'touch', 'sister', 'whisper', 'whimper', 'pain', 'soooo', 'weird', 'ms', 'jone', 'sister', 'cry', 'wish', 'breathe', 'confused', 'dad', 'super', 'mad', 'mention', 'cry', 'sleep', 'anymore', 'basement', 'floor', 'shovel']]


In [63]:
docs_list_joined = [' '.join(words) for words in data_lemmatized]

In [64]:
def display_topics(model, feature_names, no_top_words=15, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [65]:
def hyperparameter_topics(docs_list, vectorizer='count', num_topic=15,
                          binary=False, max_df=0.08, min_df=1, stop_words = 'english',
                         model='lda', no_top_words = 15):
    print('\n\n', model, vectorizer, f'num_topic:{num_topic}', f'binary:{binary}' , f'max_df:{max_df}', f'min_df:{min_df}')
    vec = None
    if vectorizer == 'count':
        vec = CountVectorizer(stop_words=stop_words, binary=binary, max_df=max_df, min_df=min_df)
#         tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
#                                 stop_words = 'english',
#                                 lowercase = True,
#                                 token_pattern = r'\b[a-zA-Z]{3,}\b',
#                                 max_df = 0.5, 
#                                 min_df = 10)
    elif vectorizer == 'tfidf':
        vec = TfidfVectorizer(stop_words=stop_words, binary=binary, max_df=max_df, min_df=min_df)
        
        
    doc_word_matrix = vec.fit_transform(docs_list)
    feature_names = vec.get_feature_names()
    
    doc_topic = None
    mod = None
    if model == 'lda':
        # Convert sparse matrix of counts to a gensim corpus
        corpus = matutils.Sparse2Corpus(doc_word_matrix.transpose())
        # We need to save a mapping (dict) of row id to word (token) for later use by gensim:
        id2word = dict((v, k) for k, v in vec.vocabulary_.items())
        mod = LatentDirichletAllocation(n_components=num_topic,random_state=RANDOM_STATE)
        mod.fit(doc_word_matrix)
        
        #pprint.pprint(mod.print_topics(num_words=20))

    elif model == 'lsa':
        mod = TruncatedSVD(num_topic,random_state=RANDOM_STATE)
        doc_topic = mod.fit_transform(doc_word_matrix)
        print('explained_variance_ratio_', lsa.explained_variance_ratio_)
        display_topics(mod, feature_names, no_top_words)
    elif model == 'nmf':
        mod = NMF(num_topic,random_state=RANDOM_STATE)
        doc_topic = mod.fit_transform(doc_word_matrix)
        display_topics(mod, feature_names, no_top_words)
        
    return (mod, vec, feature_names, doc_word_matrix, doc_topic)

### NMF

#### TFIDF

In [66]:
#num_topic=5
# min_df = 5
#max_df = .5
no_top_words = 20

for num_topic in [5, 10, 15, 20, 25]: 
    for vectorizer in ['tfidf']: 
        for binary in [True]:
            for max_df in [.2,.5]:
                for min_df in [5, 15]:
                    model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                                                 vectorizer=vectorizer, 
                                                                                                 binary=binary, 
                                                                                                 max_df=max_df, 
                                                                                                 min_df=min_df, 
                                                                                                 stop_words = stop_words,
                                                                                                 model='nmf',
                                                                                                 num_topic=num_topic,
                                                                                                 no_top_words = no_top_words
                                                                                                )

       




 nmf tfidf num_topic:5 binary:True max_df:0.2 min_df:5

Topic  0
lip, tooth, grip, cheek, lean, lift, sharp, knee, slide, gasp, soft, bone, flesh, wet, nose, breathe, swallow, wrap, gaze, inch

Topic  1
text, freak, apartment, camera, screen, computer, type, click, definitely, creepy, hallway, tonight, laptop, upstairs, stair, movie, tomorrow, downstairs, couch, random

Topic  2
doctor, patient, attempt, learn, contact, subject, victim, effect, choose, perhaps, remove, truth, discover, truly, exist, murder, escape, suffer, trust, contain

Topic  3
baby, sister, dinner, daughter, brother, sad, hug, sweet, kiss, angry, upset, daddy, bear, beautiful, favorite, fun, perfect, husband, funny, mommy

Topic  4
tree, mile, forest, animal, distance, direction, path, sun, flashlight, dirt, surround, sky, yard, wind, truck, rock, mountain, pack, hill, branch


 nmf tfidf num_topic:5 binary:True max_df:0.2 min_df:15

Topic  0
lip, tooth, grip, cheek, lean, lift, sharp, knee, slide, gasp, soft, fl


Topic  0
breath, whisper, voice, darkness, ear, silence, gasp, shoulder, glance, grip, lean, gaze, chest, shake, echo, shadow, fear, slowly, slow, sigh

Topic  1
sleep, wake, bed, asleep, awake, bedroom, weird, bathroom, dream, freak, noise, scared, completely, scar, normal, dark, usually, figure, woke, nightmare

Topic  2
patient, perhaps, begin, attempt, doctor, discover, subject, learn, effect, exist, explain, victim, truly, suffer, choose, destroy, remove, truth, condition, none

Topic  3
mom, dad, cry, kid, sister, baby, girl, boy, brother, daughter, sad, anymore, daddy, hurt, bear, upset, hug, mommy, nice, smile

Topic  4
tree, wood, forest, mile, distance, path, animal, sun, sky, direction, wind, dirt, mountain, flashlight, surround, camp, foot, edge, rock, clearing

Topic  5
drink, seat, table, smile, laugh, nod, bar, finish, reply, store, nice, lunch, parking, shrug, apartment, wear, restaurant, conversation, pocket, dinner

Topic  6
stair, kitchen, lock, upstairs, bedroom, b


Topic  0
nod, sigh, lean, glance, seat, smile, pause, shoulder, whisper, nodded, voice, shrug, shake, chuckle, gaze, reply, tone, expression, frown, answer

Topic  1
freak, weird, creepy, ill, friend, scared, seriously, crazy, definitely, answer, honestly, yesterday, scar, explain, fuck, hell, mention, tomorrow, figure, freaking

Topic  2
soul, fear, truth, truly, exist, perhaps, earth, accept, existence, die, desire, presence, god, choose, evil, reality, imagine, grow, horror, suffer

Topic  3
mom, dad, cry, kid, sister, baby, brother, girl, boy, daddy, sad, daughter, hurt, anymore, mommy, bear, upset, grow, hug, smile

Topic  4
wake, asleep, sleep, bed, dream, awake, bedroom, nightmare, lie, tired, woke, bathroom, dark, slowly, fear, completely, lay, tonight, terrified, notice

Topic  5
tree, forest, wood, mile, distance, sky, path, direction, sun, camp, rock, mountain, wind, branch, foot, surround, edge, darkness, clearing, dirt

Topic  6
stair, kitchen, upstairs, lock, hallway, do


Topic  0
nod, sigh, lean, nodded, whisper, pause, smile, voice, shoulder, glance, shrug, shake, chuckle, sorry, laugh, frown, expression, answer, shout, speak

Topic  1
freak, weird, fuck, text, fucking, tonight, friend, seriously, ill, crazy, creepy, apartment, freaking, hell, girlfriend, scared, boyfriend, totally, tomorrow, definitely

Topic  2
location, station, locate, attempt, contact, radio, arrive, investigate, enter, transcript, identify, occur, weapon, equipment, base, begin, soldier, subject, discover, remove

Topic  3
baby, daughter, daddy, kiss, cry, girl, mommy, beautiful, hug, sweet, hair, boy, pregnant, smile, hurt, perfect, husband, giggle, bed, dress

Topic  4
tree, forest, wood, path, mile, branch, distance, clearing, camp, flashlight, hike, animal, dirt, direction, rock, sun, deer, edge, mountain, leaf

Topic  5
driver, seat, parking, truck, ride, passenger, highway, vehicle, mile, speed, tire, driving, brake, window, engine, headlight, wheel, rear, radio, backseat


Topic  0
sigh, lean, pause, shrug, glance, nodded, chuckle, frown, expression, gaze, grin, gasp, lip, softly, wave, interrupt, gesture, shout, pocket, seat

Topic  1
consider, apparently, mention, obviously, honestly, doubt, assume, conversation, absolutely, clearly, concern, surprise, odd, disturb, usual, honest, agree, definitely, chat, bother

Topic  2
soul, existence, accept, truly, earth, truth, desire, exist, choose, evil, suffer, forever, presence, fade, perhaps, reality, escape, fate, powerful, destroy

Topic  3
baby, daddy, daughter, mommy, kiss, hug, beautiful, sweet, sad, honey, crying, favorite, pregnant, toy, upset, bear, giggle, perfect, husband, sweetie

Topic  4
tree, forest, path, mile, branch, distance, camp, hike, dirt, clearing, sun, rock, direction, wind, animal, sky, mountain, leaf, bush, deer

Topic  5
hallway, awake, footstep, hall, freeze, shadow, breathing, creak, nightmare, louder, movement, closer, faint, slam, tired, slow, quietly, blanket, pound, closed




Topic  0
nod, sigh, lean, nodded, smile, shrug, shoulder, shake, pause, frown, chuckle, whisper, glance, expression, sorry, chair, lip, reply, seat, answer

Topic  1
freak, weird, creepy, ill, seriously, scared, crazy, friend, scar, honestly, definitely, yesterday, fuck, freaking, explain, tomorrow, advice, mention, totally, text

Topic  2
attempt, locate, subject, begin, location, occur, source, assume, remove, enter, effect, appear, contact, base, inform, experiment, equipment, contain, arrive, extremely

Topic  3
eat, dinner, cook, kitchen, meal, hungry, plate, meat, breakfast, table, cooking, dish, bowl, taste, delicious, egg, lunch, chicken, fridge, restaurant

Topic  4
darkness, glow, shadow, grip, gaze, echo, terror, breath, illuminate, dim, heavy, gasp, grasp, peer, frame, shriek, flicker, sharp, stretch, stumble

Topic  5
tree, forest, wood, path, mile, camp, clearing, branch, hike, distance, flashlight, animal, dirt, sun, deer, rock, bush, trail, direction, mountain

Topic  

## Full Data

In [ ]:
df_full = pd.read_pickle('/Users/kirill.kogan/Documents/data/reddit/reddit_df_clean.pkl')

In [ ]:
data_full_docs = df_full['selftext'].tolist()

In [ ]:
data_full_lemmatized = process_data(data_full_docs)

In [ ]:
docs_full_list_joined = [' '.join(words) for words in data_full_lemmatized]

In [ ]:
# run on model

## LDA

In [44]:
num_topic=13
min_df = 5
max_df = .5
no_top_words = 20

In [ ]:
num_topic=10
min_df = 5
max_df = .5
no_top_words = 20

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=True, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=True, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )



In [29]:
num_topic=15
min_df = 5
max_df = .5
no_top_words = 20

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=True, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=True, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )




Topic  0
blood, arm, smile, voice, skin, mouth, pain, finger, tear, foot, tooth, word, shake, hair, leg, shoulder, floor, lip, smell, cover

Topic  1
dad, sister, mum, boy, kid, cry, monster, hurt, daddy, cabin, memory, aunt, flora, matt, window, sorry, bed, grandpa, farm, quiet

Topic  2
bed, sleep, wake, dream, bedroom, asleep, awake, window, dark, apartment, nightmare, noise, bathroom, closet, lie, tired, boyfriend, figure, fear, sleep_paralysis

Topic  3
tree, wood, forest, truck, cabin, trail, tent, mile, window, camp, lake, path, foot, josh, mountain, dark, branch, creature, rock, flashlight

Topic  4
fuck, text, picture, apartment, friend, weird, write, fucking, answer, kid, page, ill, reply, hell, girlfriend, girl, cop, letter, miss, kill

Topic  5
daughter, girl, doll, baby, sister, daddy, kid, mommy, boy, cry, uncle, husband, aunt, grandmother, miss, hair, grandma, mama, dress, beautiful

Topic  6
mom, grandma, letter, kid, uncle, kitchen, cry, grandmother, aunt, smile, sist

In [30]:
num_topic=20
min_df = 5
max_df = .5
no_top_words = 20

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=True, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=True, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )

model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )



Topic  0
nod, sigh, whisper, lean, voice, pause, glance, shoulder, shake, shout, nodded, silence, smile, expression, gasp, breath, softly, shrug, speak, frown

Topic  1
freak, weird, friend, text, fuck, creepy, seriously, ill, crazy, fucking, honestly, girlfriend, answer, definitely, conversation, joke, hell, yesterday, scared, explain

Topic  2
soul, fear, earth, exist, perhaps, truly, presence, truth, existence, accept, desire, reality, god, choose, die, imagine, evil, horror, escape, grow

Topic  3
mom, dad, kid, cry, sister, baby, girl, boy, brother, daddy, daughter, sad, sometimes, anymore, hurt, bear, mommy, grow, upset, miss

Topic  4
tree, forest, wood, mile, path, animal, distance, branch, sun, camp, flashlight, clearing, sky, edge, direction, rock, wind, dirt, foot, hike

Topic  5
driver, seat, parking, truck, ride, passenger, vehicle, mile, highway, radio, speed, window, engine, station, driving, wheel, brake, exit, route, headlight

Topic  6
stair, upstairs, kitchen, lock,

#### Count


Topic  0
nod, smile, sigh, shoulder, lean, laugh, seat, shake, glance, sorry, table, reply, shrug, chair, pause, grin, answer, grab, wave, pick

Topic  1
weird, fuck, hell, friend, figure, freak, fucking, crazy, definitely, sort, suppose, creepy, nice, ill, honestly, normal, usually, explain, swear, exactly

Topic  2
explain, doctor, eventually, completely, exactly, manage, assume, clearly, learn, notice, patient, attempt, somehow, begin, consider, contact, apparently, strange, sort, mention

Topic  3
tree, wood, mile, foot, forest, distance, direction, dark, fast, animal, sun, deep, path, edge, sky, spot, dirt, wind, surround, window

Topic  4
sleep, wake, bed, asleep, awake, dream, lie, bedroom, tired, lay, bathroom, nightmare, sometimes, woke, window, pass, normal, usual, pillow, figure

Topic  5
mom, kid, dad, cry, girl, boy, miss, baby, hurt, sometimes, sister, brother, anymore, grow, bear, daughter, smile, tear, learn, nice

Topic  6
skin, smell, mouth, hair, blood, eat, wear, c


Topic  0
blood, arm, mouth, smile, skin, foot, pain, finger, tooth, floor, tear, shake, hair, fill, shoulder, push, leg, cover, grab, cry

Topic  1
dad, brother, cry, sister, grandfather, bed, aunt, uncle, monster, mum, hurt, wood, window, sorry, alone, gun, farm, tear, bedroom, memory

Topic  2
bed, sleep, wake, dream, bedroom, asleep, dark, awake, lie, bathroom, figure, cry, apartment, hair, fear, nightmare, sister, normal, sometimes, baby

Topic  3
tree, wood, forest, foot, dark, trail, path, leg, creature, deep, spider, josh, branch, jr, lake, rock, mile, cold, edge, catch

Topic  4
kid, boy, laugh, smile, teacher, perfect, rob, eat, doll, daddy, ghost, weird, brother, miss, cry, nice, friend, sometimes, baby, camp

Topic  5
paper, write, word, picture, answer, doctor, page, notice, box, patient, explain, lie, speak, file, sort, kill, figure, letter, desk, computer

Topic  6
mom, sister, smile, brother, basement, eat, baby, letter, shake, friend, uncle, laugh, bus, arm, leg, cry, 

### LDA


#### TFIDF

In [33]:
model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='lda',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )
pyLDAvis.sklearn.prepare(model, doc_words_matrix, vec)

//anaconda3/envs/reddit/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.211318  0.003964  1       1        87.784022
9      0.170205  0.005846  2       1        4.543280 
5      0.006925 -0.024557  3       1        0.800463 
6      0.004737 -0.006890  4       1        0.741122 
3      0.004734 -0.003138  5       1        0.731229 
4      0.004505 -0.001482  6       1        0.707790 
12     0.003863 -0.000910  7       1        0.706063 
0      0.003490  0.001419  8       1        0.689672 
1      0.003018  0.004094  9       1        0.670782 
2      0.002692  0.004625  10      1        0.662964 
11     0.002351  0.005243  11      1        0.656574 
10     0.002453  0.005614  12      1        0.654988 
8      0.002345  0.006173  13      1        0.651053 , topic_info=      Category       Freq                Term      Total  loglift  logprob
7167   Default  25.000000  loud                25.000000  30.0000  30.0000
5132   Default  25.000000  glass               25.000000  29.0000  29.0000
9944   Default  25.000000  reply               25.000000  28.0000  28.0000
5485   Default  25.000000  hallway             25.000000  27.0000  27.0000
3765   Default  25.000000  ear                 25.000000  26.0000  26.0000
5726   Default  25.000000  hide                25.000000  25.0000  25.0000
7655   Default  25.000000  mirror              25.000000  24.0000  24.0000
4528   Default  25.000000  fill                25.000000  23.0000  23.0000
3951   Default  26.000000  empty               26.000000  22.0000  22.0000
6659   Default  25.000000  jump                25.000000  21.0000  21.0000
9487   Default  26.000000  push                26.000000  20.0000  20.0000
975    Default  25.000000  begin               25.000000  19.0000  19.0000
9532   Default  25.000000  quiet               25.000000  18.0000  18.0000
13751  Default  25.000000  wonder              25.000000  17.0000  17.0000
12307  Default  26.000000  text                26.000000  16.0000  16.0000
5966   Default  26.000000  hurt                26.000000  15.0000  15.0000
6764   Default  24.000000  knife               24.000000  14.0000  14.0000
13622  Default  27.000000  whisper             27.000000  13.0000  13.0000
5396   Default  27.000000  grow                27.000000  12.0000  12.0000
11332  Default  27.000000  sorry               27.000000  11.0000  11.0000
8074   Default  24.000000  nice                24.000000  10.0000  10.0000
3654   Default  24.000000  drink               24.000000  9.0000   9.0000 
462    Default  27.000000  anymore             27.000000  8.0000   8.0000 
10832  Default  24.000000  shoulder            24.000000  7.0000   7.0000 
4775   Default  27.000000  forget              27.000000  6.0000   6.0000 
5842   Default  24.000000  hop                 24.000000  5.0000   5.0000 
1828   Default  24.000000  chair               24.000000  4.0000   4.0000 
4136   Default  24.000000  eventually          24.000000  3.0000   3.0000 
8138   Default  27.000000  normal              27.000000  2.0000   2.0000 
1742   Default  24.000000  cat                 24.000000  1.0000   1.0000 
7738   Topic1   71.022064  mom                 71.262712  0.1269  -6.0953 
946    Topic1   70.334748  bed                 70.575396  0.1269  -6.1051 
2883   Topic1   67.981993  dad                 68.222642  0.1268  -6.1391 
11063  Topic1   59.284218  sleep               59.524867  0.1262  -6.2760 
5115   Topic1   58.914635  girl                59.155284  0.1262  -6.2822 
13398  Topic1   58.882886  voice               59.123535  0.1262  -6.2828 
13696  Topic1   56.542502  window              56.783151  0.1260  -6.3233 
6715   Topic1   53.848464  kid                 54.089113  0.1258  -6.3721 
11163  Topic1   52.880763  smile               53.121412  0.1258  -6.3903 
4664   Topic1   48.441037  floor               48.681686  0.1253  -6.4780 
4925   Topic1   47.074754  fuck                47.315403  0.1252  -6.506

In [34]:
model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=True, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='lda',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )
pyLDAvis.sklearn.prepare(model, doc_words_matrix, vec)

//anaconda3/envs/reddit/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.227520 -0.041190  1       1        76.087874
9     -0.093636 -0.156235  2       1        16.811540
6     -0.045669  0.090158  3       1        1.318760 
12    -0.015482  0.023508  4       1        0.701682 
0     -0.013707  0.019996  5       1        0.666068 
5     -0.010413  0.013576  6       1        0.607400 
1     -0.008093  0.009357  7       1        0.566220 
4     -0.007829  0.008703  8       1        0.560343 
3     -0.007810  0.008841  9       1        0.559550 
2     -0.007014  0.007173  10      1        0.543566 
11    -0.006692  0.006756  11      1        0.538632 
10    -0.005639  0.004752  12      1        0.519967 
8     -0.005535  0.004606  13      1        0.518398 , topic_info=      Category       Freq           Term      Total  loglift  logprob
11421  Default  21.000000  spend          21.000000  30.0000  30.0000
5822   Default  19.000000  honestly       19.000000  29.0000  29.0000
4841   Default  23.000000  freak          23.000000  28.0000  28.0000
13599  Default  18.000000  whenever       18.000000  27.0000  27.0000
7889   Default  19.000000  murder         19.000000  26.0000  26.0000
...        ...        ...     ...               ...      ...      ...
8945   Topic13  0.016631   pitter_patter  0.585095   1.7017  -9.5449 
3417   Topic13  0.016631   disorientate   0.502993   1.8529  -9.5449 
3992   Topic13  0.016631   engaging       0.487860   1.8834  -9.5449 
7225   Topic13  0.016631   lurked         0.705296   1.5148  -9.5449 
5792   Topic13  0.016631   hog            0.632844   1.6232  -9.5449 

[545 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
207    2      1.086042  affirmation    
327    1      0.942148  alone          
327    2      0.053837  alone          
409    2      0.925633  angrily        
436    1      0.953867  answer         
436    2      0.054507  answer         
462    1      0.939533  anymore        
462    2      0.030308  anymore        
551    1      0.938099  arm            
551    2      0.080408  arm            
566    2      0.944760  arrival        
601    1      0.955093  asleep         
601    2      0.035374  asleep         
631    1      0.976750  assume         
631    2      0.040698  assume         
704    3      0.913305  austin         
724    2      0.894782  avail          
736    2      0.858663  await          
894    1      0.978743  bathroom       
894    2      0.037644  bathroom       
946    1      0.945800  bed            
946    2      0.046137  bed            
1283   1      0.935074  bootleg        
1299   1      0.986135  bother         
1299   2      0.046959  bother         
1444   1      0.950357  brother        
1444   2      0.043198  brother        
1750   1      0.909303  catch          
1750   2      0.085247  catch          
1797   1      1.007669  cell           
1935   2      0.956548  chin           
1990   1      0.349358  cigarette      
1990   2      0.582263  cigarette      
2251   3      1.065777  comforter      
2311   1      0.929426  completely     
2311   2      0.059963  completely     
2472   1      0.975918  contact        
2529   1      0.978426  convince       
2541   2      0.944028  cooking        
2629   1      0.913359  cover          
2629   2      0.085627  cover          
2686   1      0.956276  crazy          
2686   2      0.036780  crazy          
2687   1      0.949383  creak          
2713   1      0.960059  creepy         
2733   2      0.835444  crimson        
2796   1      0.934145  cry            
2796   2      0.054950  cry            
2920   1      0.926106  dark           
2920   2      0.050060  dark           
3079   2      0.952097  delicious      
3391   3      0.908025  dishwasher     
3743   1      0.171023  dusty          
3743   2      0.684093  dusty          
3782   3      1.075227  earrin

#### Count

In [35]:
model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='lda',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )
pyLDAvis.sklearn.prepare(model, doc_words_matrix, vec)

//anaconda3/envs/reddit/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.017062 -0.070255  1       1        18.843636
7      0.022849 -0.036844  2       1        15.506641
12    -0.093786 -0.010925  3       1        13.033937
11     0.080207 -0.168521  4       1        7.093204 
5      0.013042  0.037487  5       1        6.897221 
8     -0.120009  0.002394  6       1        6.229155 
2     -0.035195  0.071693  7       1        6.161689 
0      0.087839  0.021499  8       1        6.154227 
9      0.000974  0.014768  9       1        5.774869 
1     -0.133275 -0.004866  10      1        5.439270 
4      0.107052  0.056734  11      1        5.253953 
10     0.056319  0.124072  12      1        1.810214 
3      0.031044 -0.037235  13      1        1.801985 , topic_info=      Category         Freq   Term        Total  loglift  logprob
7738   Default  3517.000000  mom    3517.000000  30.0000  30.0000
2883   Default  3485.000000  dad    3485.000000  29.0000  29.0000
3516   Default  1699.000000  dog    1699.000000  28.0000  28.0000
12688  Default  1728.000000  tree   1728.000000  27.0000  27.0000
13760  Default  1825.000000  wood   1825.000000  26.0000  26.0000
...        ...          ...   ...           ...      ...      ...
762    Topic13  61.953849    baby   1321.738362  0.9560  -5.7856 
11163  Topic13  72.008907    smile  3743.271552  0.0654  -5.6352 
11383  Topic13  58.624121    speak  2216.685977  0.3837  -5.8409 
3021   Topic13  53.442305    deep   1706.257594  0.5528  -5.9334 
13515  Topic13  52.057226    wave   868.055240   1.2024  -5.9597 

[1035 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
8      1      0.070560  abby
8      2      0.105840  abby
8      5      0.132300  abby
8      6      0.035280  abby
8      10     0.617400  abby
...   ..           ...   ...
13894  2      0.078823  zach
13894  5      0.019706  zach
13894  10     0.886764  zach
13895  6      0.781818  zack
13895  10     0.142149  zack

[4740 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 13, 12, 6, 9, 3, 1, 10, 2, 5, 11, 4])

In [36]:
model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=False, 
                                                                         max_df=max_df, 
                                                                         min_df=min_df, 
                                                                         stop_words = stop_words,
                                                                         model='lda',
                                                                         num_topic=num_topic,
                                                                         no_top_words = no_top_words
                                                                        )
pyLDAvis.sklearn.prepare(model, doc_words_matrix, vec)

//anaconda3/envs/reddit/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.017062 -0.070255  1       1        18.843636
7      0.022849 -0.036844  2       1        15.506641
12    -0.093786 -0.010925  3       1        13.033937
11     0.080207 -0.168521  4       1        7.093204 
5      0.013042  0.037487  5       1        6.897221 
8     -0.120009  0.002394  6       1        6.229155 
2     -0.035195  0.071693  7       1        6.161689 
0      0.087839  0.021499  8       1        6.154227 
9      0.000974  0.014768  9       1        5.774869 
1     -0.133275 -0.004866  10      1        5.439270 
4      0.107052  0.056734  11      1        5.253953 
10     0.056319  0.124072  12      1        1.810214 
3      0.031044 -0.037235  13      1        1.801985 , topic_info=      Category         Freq   Term        Total  loglift  logprob
7738   Default  3517.000000  mom    3517.000000  30.0000  30.0000
2883   Default  3485.000000  dad    3485.000000  29.0000  29.0000
3516   Default  1699.000000  dog    1699.000000  28.0000  28.0000
12688  Default  1728.000000  tree   1728.000000  27.0000  27.0000
13760  Default  1825.000000  wood   1825.000000  26.0000  26.0000
...        ...          ...   ...           ...      ...      ...
762    Topic13  61.953849    baby   1321.738362  0.9560  -5.7856 
11163  Topic13  72.008907    smile  3743.271552  0.0654  -5.6352 
11383  Topic13  58.624121    speak  2216.685977  0.3837  -5.8409 
3021   Topic13  53.442305    deep   1706.257594  0.5528  -5.9334 
13515  Topic13  52.057226    wave   868.055240   1.2024  -5.9597 

[1035 rows x 6 columns], token_table=       Topic      Freq  Term
term                        
8      1      0.070560  abby
8      2      0.105840  abby
8      5      0.132300  abby
8      6      0.035280  abby
8      10     0.617400  abby
...   ..           ...   ...
13894  2      0.078823  zach
13894  5      0.019706  zach
13894  10     0.886764  zach
13895  6      0.781818  zack
13895  10     0.142149  zack

[4740 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 8, 13, 12, 6, 9, 3, 1, 10, 2, 5, 11, 4])

### Misc

In [40]:
model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='count', 
                                                                         binary=False, 
                                                                         max_df=0.5, 
                                                                         min_df=5, 
                                                                         stop_words = stop_words,
                                                                         model='lda',
                                                                         num_topic=15,
                                                                         no_top_words = no_top_words
                                                                        )



Topic  0
skin, chest, screamed, feet, flesh, fingers, mouth, throat, pain, blood, legs, arms, leaned, pulled, ears

Topic  1
fuck, shit, fucking, dude, alright, yeah, gon, na, hell, guys, damn, fucked, sorry, weird, ass

Topic  2
fully, truth, particular, patient, begin, returned, knowledge, certainly, caused, fear, dr, effects, contact, imagine, believed

Topic  3
locked, doors, shut, hallway, window, lights, kitchen, windows, walk, hearing, lock, update, bedroom, bathroom, closed

Topic  4
sleep, asleep, awake, sleeping, wake, woke, slept, waking, woken, sleeps, covers, bedroom, scared, lay, shadow

Topic  5
trees, woods, forest, trail, tree, sun, dirt, wind, sky, path, leaves, trip, rocks, hill, distance

Topic  6
asks, pulls, stops, sits, leans, smiles, starts, feels, opens, ta, kinda, grab, tongue, nods, everybody

Topic  7
smiled, answered, nodded, parking, smile, sighed, seat, dressed, oh, handed, glanced, replied, shrugged, suit, hello

Topic  8
crying, mom, downstairs, cried,

In [62]:
model, vec, feature_names, doc_words_matrix, doc_topic = hyperparameter_topics(docs_list_joined, 
                                                                         vectorizer='tfidf', 
                                                                         binary=True, 
                                                                         max_df=0.5, 
                                                                         min_df=5, 
                                                                         stop_words = stop_words,
                                                                         model='nmf',
                                                                         num_topic=15,
                                                                         no_top_words = 15
                                                                        )



Topic  0
skin, chest, screamed, pain, feet, fingers, blood, throat, arms, mouth, lips, neck, legs, leaned, flesh

Topic  1
fuck, shit, fucking, dude, gon, alright, yeah, na, hell, damn, fucked, sorry, crazy, guys, weird

Topic  2
existence, truth, terror, presence, creature, earth, fear, simply, terrible, sight, save, brief, survive, returned, surface

Topic  3
locked, doors, shut, hallway, window, hearing, lights, walk, kitchen, windows, update, closed, lock, bedroom, staring

Topic  4
sleep, awake, asleep, sleeping, wake, woke, slept, woken, waking, sleeps, covers, lay, bedroom, nights, noise

Topic  5
trees, woods, forest, tree, trail, sun, dirt, wind, leaves, rocks, path, sky, trip, hill, branches

Topic  6
asks, stops, pulls, leans, sits, smiles, starts, feels, opens, ta, kinda, grab, wear, turns, nods

Topic  7
downstairs, crying, mom, upstairs, ran, cried, screaming, couch, stairs, scared, terrified, stopped, loud, scream, tears

Topic  8
basement, box, written, flashlight, tab

In [24]:
pprint.pprint(doc_topic)

array([[0.00802479, 0.        , 0.        , ..., 0.        , 0.23093307,
        0.        ],
       [0.01849226, 0.00713915, 0.        , ..., 0.        , 0.2381418 ,
        0.00096101],
       [0.        , 0.0004484 , 0.09312921, ..., 0.        , 0.10138511,
        0.        ],
       ...,
       [0.        , 0.09336669, 0.        , ..., 0.        , 0.        ,
        0.02529162],
       [0.        , 0.08309804, 0.00838545, ..., 0.04809558, 0.        ,
        0.01768782],
       [0.        , 0.01286271, 0.        , ..., 0.02252478, 0.        ,
        0.        ]])


In [42]:
topic_weights_df = pd.DataFrame(doc_topic)

In [43]:
topic_weights_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.000000,0.007233,0.000000,0.050579,0.000000,0.000000,0.031865,0.000000,0.128993,0.007452,0.116160,0.002031,0.000000,0.000000,0.000000
1,0.032406,0.067545,0.000000,0.109404,0.000000,0.001261,0.000000,0.000000,0.000000,0.032685,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.049796,0.120924,0.036486,0.016707,0.000000,0.000000,0.000000,0.000000,0.008548,0.000000,0.000000,0.016075,0.065149
3,0.000000,0.000000,0.000000,0.023088,0.000000,0.000000,0.000000,0.000978,0.069347,0.082375,0.000000,0.000000,0.017134,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.002533,0.000000,0.000000,0.000000,0.000000,0.026967,0.009869,0.077078,0.000897,0.108176,0.000000,0.024159


In [44]:
df_sample.merge(topic_weights_df)

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [ ]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
#pyLDAvis.sklearn.prepare(model, corpus, words)
#https://github.com/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb

pyLDAvis.sklearn.prepare(model, doc_words_matrix, vec)

### Non-negative matrix factorization

### Latent Dirichlet Allocation

In [ ]:
#new TF-IDF Vectorizer
cv_tfidf = TfidfVectorizer(stop_words=stopword_list,max_df=0.08,min_df=0.004)
# cv_tfidf = TfidfVectorizer(stop_words=stopword_list,max_df=0.6,min_df=0.005,ngram_range=(1, 2))
doc_word = cv_tfidf.fit_transform(df_news_clean['title_intro_clean'])
print(doc_word.shape)
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word.transpose())
# We need to save a mapping (dict) of row id to word (token) for later use by gensim:
id2word = dict((v, k) for k, v in cv_tfidf.vocabulary_.items())
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaMulticore(corpus=corpus, num_topics=13, id2word=id2word, passes=5, workers=1)
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]
# View Topic Distribution
doc_cluster = [sorted(doc,key=lambda row:row[1])[::-1][0][0] for doc in lda_docs]
sns.barplot(x=pd.Series(doc_cluster).value_counts().keys(),y=pd.Series(doc_cluster).value_counts().values);
# # Check out the document vectors in the topic space for the first 5 documents
# lda_docs[:5]
lda.print_topics()
# #Save Trained Model
# with open('lda.pkl', 'wb') as picklefile:
#     pickle.dump(lda, picklefile)





In [ ]:
# def lda_docs(doc_list):
#     # A mapping of word IDs to words.
#     words = corpora.Dictionary(doc_list)

#     # Turns each document into a bag of words.
#     corpus = [words.doc2bow(doc) for doc in doc_list]
    
#     lda_model = gensim.models.ldamodel.LdaMulticore(corpus=corpus,
#                                                    id2word=words,
#                                                    num_topics=10, 
#                                                    random_state=RANDOM_STATE,
#                                                    update_every=1,
#                                                    passes=10,
#                                                    alpha='auto',
#                                                    per_word_topics=True,
#                                                    workers=W)
#     return lda_model

#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
pyLDAvis.gensim.prepare(model, corpus, words)

In [ ]:

#lda_model_a = lda_docs(process_docs(docs_a, base_nlp))


### Correlation Explanation

# Layers

## Over time

## Segment? (prob no)

## Character?

# Explore

## pyldavis

## scattertext

# Other

## sentiment

## word2vec

In [ ]:
def transform_texts(texts):
    # Load the annotation models
    nlp = English()
    # Stream texts through the models. We accumulate a buffer and release
    # the GIL around the parser, for efficient multi-threading.
    for doc in nlp.pipe(texts, n_threads=4):
        # Iterate over base NPs, e.g. "all their good ideas"
        for np in doc.noun_chunks:
            # Only keep adjectives and nouns, e.g. "good ideas"
            while len(np) > 1 and np[0].dep_ not in ('amod', 'compound'):
                np = np[1:]
            if len(np) > 1:
                # Merge the tokens, e.g. good_ideas
                np.merge(np.root.tag_, np.text, np.root.ent_type_)
            # Iterate over named entities
            for ent in doc.ents:
                if len(ent) > 1:
                    # Merge them into single tokens
                    ent.merge(ent.root.tag_, ent.text, ent.label_)
        token_strings = []
        for token in tokens:
            text = token.text.replace(' ', '_')
            tag = token.ent_type_ or token.pos_
            token_strings.append('%s|%s' % (text, tag))
        yield ' '.join(token_strings)

## rec system

## clustering

# Results

## Plotly/Tableau/D3/Flask

## Gensim LDA

In [173]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:2]]

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:built Dictionary(28590 unique tokens: ['abuse', 'abusive', 'afterwards', 'agree', 'alley']...) from 1600 documents (total 726841 corpus positions)


[[('abuse', 1),
  ('abusive', 2),
  ('afterwards', 1),
  ('agree', 1),
  ('alley', 1),
  ('angry', 1),
  ('answer', 1),
  ('anyways', 1),
  ('apparently', 1),
  ('appear', 1),
  ('arm', 1),
  ('arrest', 1),
  ('assume', 1),
  ('attic', 1),
  ('awhile', 2),
  ('bad', 1),
  ('baseball', 2),
  ('baseball_bat', 1),
  ('bath', 1),
  ('bathroom', 2),
  ('bathtub', 2),
  ('be', 3),
  ('beat', 1),
  ('bed', 1),
  ('begin', 1),
  ('belly', 1),
  ('bill', 1),
  ('bitch', 1),
  ('blame', 1),
  ('blend', 1),
  ('bloated', 1),
  ('blonde_hair', 1),
  ('bob', 1),
  ('bother', 1),
  ('box', 1),
  ('break', 2),
  ('breast', 1),
  ('brief', 1),
  ('burglar', 1),
  ('buster', 1),
  ('cabinet', 1),
  ('can', 1),
  ('capillary', 1),
  ('carpet', 1),
  ('carry', 1),
  ('cat', 5),
  ('cereal', 1),
  ('chair', 1),
  ('cheap', 1),
  ('cheer', 1),
  ('chest', 3),
  ('chested', 1),
  ('chin', 1),
  ('choice', 1),
  ('chuckle', 1),
  ('claw', 1),
  ('close', 1),
  ('closet', 1),
  ('clothe', 2),
  ('color', 1),


In [174]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


INFO:gensim.models.ldamodel:using autotuned alpha, starting with [0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667, 0.06666667]
INFO:gensim.models.ldamodel:using symmetric eta at 0.06666666666666667
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamodel:running online (multi-pass) LDA training, 15 topics, 10 passes over the supplied corpus of 1600 documents, updating model once every 100 documents, evaluating perplexity every 1000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #100/1600
INFO:gensim.models.ldamodel:optimized alpha [0.101912424, 0.11223675, 0.117009796, 0.10759729, 0.11878459, 0.11327178, 0.11795741, 0.11512109, 0.09948169, 0.09575366, 0.084208086, 0.11701481, 0.11169133, 0.10386495, 0.0953114]
INFO:gensim.models.ldamodel:merging changes fro

INFO:gensim.models.ldamodel:topic #9 (0.041): 0.003*"eye" + 0.002*"box" + 0.002*"christ" + 0.002*"know" + 0.001*"look" + 0.001*"walk" + 0.001*"floor" + 0.001*"be" + 0.001*"not" + 0.001*"scream"
INFO:gensim.models.ldamodel:topic #7 (0.293): 0.065*"not" + 0.034*"be" + 0.032*"do" + 0.014*"s" + 0.013*"would" + 0.011*"have" + 0.008*"eye" + 0.007*"could" + 0.007*"look" + 0.006*"girl"
INFO:gensim.models.ldamodel:topic #2 (0.433): 0.013*"eye" + 0.013*"door" + 0.008*"look" + 0.007*"know" + 0.006*"sit" + 0.006*"sound" + 0.006*"walk" + 0.006*"scream" + 0.005*"seem" + 0.005*"voice"
INFO:gensim.models.ldamodel:topic #4 (0.472): 0.019*"not" + 0.013*"door" + 0.011*"do" + 0.010*"be" + 0.009*"look" + 0.008*"know" + 0.007*"walk" + 0.006*"bed" + 0.006*"scream" + 0.005*"eye"
INFO:gensim.models.ldamodel:topic diff=0.371973, rho=0.408248
INFO:gensim.models.ldamodel:PROGRESS: pass 0, at document #700/1600
INFO:gensim.models.ldamodel:optimized alpha [0.07814045, 0.052655257, 0.54155475, 0.042877536, 0.570798,

INFO:gensim.models.ldamodel:merging changes from 100 documents into a model of 1600 documents
INFO:gensim.models.ldamodel:topic #10 (0.026): 0.000*"caller" + 0.000*"operator" + 0.000*"look" + 0.000*"not" + 0.000*"eye" + 0.000*"be" + 0.000*"door" + 0.000*"seem" + 0.000*"mom" + 0.000*"girl"
INFO:gensim.models.ldamodel:topic #9 (0.030): 0.000*"eye" + 0.000*"box" + 0.000*"christ" + 0.000*"know" + 0.000*"look" + 0.000*"walk" + 0.000*"floor" + 0.000*"be" + 0.000*"not" + 0.000*"scream"
INFO:gensim.models.ldamodel:topic #11 (0.457): 0.010*"look" + 0.008*"eye" + 0.007*"walk" + 0.006*"door" + 0.006*"mirror" + 0.005*"pull" + 0.004*"sit" + 0.004*"star" + 0.004*"fuck" + 0.004*"scream"
INFO:gensim.models.ldamodel:topic #4 (0.960): 0.015*"door" + 0.010*"not" + 0.009*"look" + 0.008*"know" + 0.008*"walk" + 0.008*"bed" + 0.007*"window" + 0.006*"sit" + 0.006*"do" + 0.006*"hear"
INFO:gensim.models.ldamodel:topic #2 (1.144): 0.013*"eye" + 0.011*"door" + 0.007*"look" + 0.007*"know" + 0.006*"scream" + 0.006*

INFO:gensim.models.ldamodel:topic diff=0.090082, rho=0.235702
INFO:gensim.models.ldamodel:PROGRESS: pass 1, at document #200/1600
INFO:gensim.models.ldamodel:optimized alpha [0.050153278, 0.03064798, 1.3458822, 0.027273381, 1.0596523, 0.48917088, 0.48908088, 0.35279593, 0.06412831, 0.024847986, 0.022342972, 0.47715527, 0.041416835, 0.041127328, 0.048726812]
INFO:gensim.models.ldamodel:merging changes from 100 documents into a model of 1600 documents
INFO:gensim.models.ldamodel:topic #10 (0.022): 0.000*"caller" + 0.000*"operator" + 0.000*"look" + 0.000*"not" + 0.000*"eye" + 0.000*"be" + 0.000*"door" + 0.000*"seem" + 0.000*"mom" + 0.000*"girl"
INFO:gensim.models.ldamodel:topic #9 (0.025): 0.000*"eye" + 0.000*"box" + 0.000*"christ" + 0.000*"know" + 0.000*"look" + 0.000*"walk" + 0.000*"floor" + 0.000*"be" + 0.000*"not" + 0.000*"scream"
INFO:gensim.models.ldamodel:topic #5 (0.489): 0.005*"seem" + 0.005*"look" + 0.005*"know" + 0.005*"patient" + 0.004*"girl" + 0.004*"grandmother" + 0.004*"woo

INFO:gensim.models.ldamodel:topic #4 (1.121): 0.019*"door" + 0.010*"bed" + 0.010*"look" + 0.008*"walk" + 0.008*"know" + 0.008*"sleep" + 0.008*"sit" + 0.007*"window" + 0.006*"hear" + 0.006*"kid"
INFO:gensim.models.ldamodel:topic #2 (1.452): 0.015*"eye" + 0.009*"door" + 0.008*"look" + 0.006*"scream" + 0.006*"know" + 0.006*"voice" + 0.005*"sound" + 0.005*"sit" + 0.005*"stand" + 0.005*"walk"
INFO:gensim.models.ldamodel:topic diff=0.054718, rho=0.235702
INFO:gensim.models.ldamodel:PROGRESS: pass 1, at document #800/1600
INFO:gensim.models.ldamodel:optimized alpha [0.045094438, 0.025836335, 1.4725653, 0.02343747, 1.1567862, 0.48630854, 0.44958982, 0.33782703, 0.060588352, 0.021443173, 0.019556897, 0.45219514, 0.03561504, 0.03723006, 0.050699208]
INFO:gensim.models.ldamodel:merging changes from 100 documents into a model of 1600 documents
INFO:gensim.models.ldamodel:topic #10 (0.020): 0.000*"caller" + 0.000*"operator" + 0.000*"look" + 0.000*"not" + 0.000*"eye" + 0.000*"be" + 0.000*"door" + 0.

...

INFO:gensim.models.ldamodel:topic #2 (2.440): 0.018*"eye" + 0.009*"look" + 0.008*"scream" + 0.006*"voice" + 0.006*"arm" + 0.006*"stand" + 0.005*"pull" + 0.005*"know" + 0.005*"sound" + 0.005*"door"
INFO:gensim.models.ldamodel:topic diff=0.019940, rho=0.200000
INFO:gensim.models.ldamodel:PROGRESS: pass 8, at document #700/1600
INFO:gensim.models.ldamodel:optimized alpha [0.035303798, 0.008374359, 2.4598534, 0.007970522, 2.3246584, 0.8197085, 0.62953323, 0.3937874, 0.07057995, 0.006971975, 0.0067624548, 0.4863049, 0.021600148, 0.023208555, 0.10285876]
INFO:gensim.models.ldamodel:merging changes from 100 documents into a model of 1600 documents
INFO:gensim.models.ldamodel:topic #10 (0.007): 0.000*"glyni" + 0.000*"icebreaker" + 0.000*"beefy" + 0.000*"donald" + 0.000*"dumbfuck" + 0.000*"wilber" + 0.000*"griffith" + 0.000*"inquisitive" + 0.000*"illegible_text" + 0.000*"tequila"
INFO:gensim.models.ldamodel:topic #9 (0.007): 0.000*"glyni" + 0.000*"icebreaker" + 0.000*"beefy" + 0.000*"donald" + 

INFO:gensim.models.ldamodel:merging changes from 100 documents into a model of 1600 documents
INFO:gensim.models.ldamodel:topic #10 (0.007): 0.000*"glyni" + 0.000*"icebreaker" + 0.000*"beefy" + 0.000*"donald" + 0.000*"dumbfuck" + 0.000*"wilber" + 0.000*"griffith" + 0.000*"inquisitive" + 0.000*"illegible_text" + 0.000*"tequila"
INFO:gensim.models.ldamodel:topic #9 (0.007): 0.000*"glyni" + 0.000*"icebreaker" + 0.000*"beefy" + 0.000*"donald" + 0.000*"dumbfuck" + 0.000*"wilber" + 0.000*"griffith" + 0.000*"inquisitive" + 0.000*"illegible_text" + 0.000*"tequila"
INFO:gensim.models.ldamodel:topic #5 (0.849): 0.008*"know" + 0.006*"seem" + 0.006*"doctor" + 0.006*"patient" + 0.004*"learn" + 0.004*"die" + 0.004*"guard" + 0.004*"girl" + 0.004*"speak" + 0.004*"write"
INFO:gensim.models.ldamodel:topic #4 (2.421): 0.022*"door" + 0.011*"look" + 0.011*"walk" + 0.010*"bed" + 0.010*"sit" + 0.009*"know" + 0.008*"sleep" + 0.008*"hear" + 0.008*"window" + 0.008*"remember"
INFO:gensim.models.ldamodel:topic #2

INFO:gensim.models.ldamodel:topic #5 (0.873): 0.008*"know" + 0.007*"seem" + 0.006*"patient" + 0.006*"doctor" + 0.004*"girl" + 0.004*"learn" + 0.004*"write" + 0.004*"die" + 0.004*"guard" + 0.004*"speak"
INFO:gensim.models.ldamodel:topic #4 (2.463): 0.023*"door" + 0.011*"look" + 0.010*"walk" + 0.009*"bed" + 0.009*"sit" + 0.009*"know" + 0.008*"sleep" + 0.008*"window" + 0.008*"hear" + 0.008*"maybe"
INFO:gensim.models.ldamodel:topic #2 (2.498): 0.017*"eye" + 0.009*"look" + 0.007*"scream" + 0.006*"arm" + 0.006*"voice" + 0.005*"pull" + 0.005*"stand" + 0.005*"know" + 0.005*"sound" + 0.005*"door"
INFO:gensim.models.ldamodel:topic diff=0.032320, rho=0.196116
INFO:gensim.models.ldamodel:PROGRESS: pass 9, at document #200/1600
INFO:gensim.models.ldamodel:optimized alpha [0.03503989, 0.007964812, 2.5332758, 0.007603659, 2.3813767, 0.852336, 0.65571874, 0.4112496, 0.072094485, 0.0065881046, 0.006400762, 0.5132752, 0.021967625, 0.022895105, 0.106934495]
INFO:gensim.models.ldamodel:merging changes fro

INFO:gensim.models.ldamodel:topic diff=0.019191, rho=0.196116
INFO:gensim.models.ldamodel:PROGRESS: pass 9, at document #700/1600
INFO:gensim.models.ldamodel:optimized alpha [0.035550572, 0.0077880677, 2.4963756, 0.0074161417, 2.39603, 0.84398085, 0.635011, 0.39551565, 0.07148591, 0.0064250664, 0.0062467763, 0.49510732, 0.021454044, 0.023062428, 0.10655086]
INFO:gensim.models.ldamodel:merging changes from 100 documents into a model of 1600 documents
INFO:gensim.models.ldamodel:topic #10 (0.006): 0.000*"glyni" + 0.000*"icebreaker" + 0.000*"beefy" + 0.000*"donald" + 0.000*"dumbfuck" + 0.000*"wilber" + 0.000*"griffith" + 0.000*"inquisitive" + 0.000*"illegible_text" + 0.000*"tequila"
INFO:gensim.models.ldamodel:topic #9 (0.006): 0.000*"glyni" + 0.000*"icebreaker" + 0.000*"beefy" + 0.000*"donald" + 0.000*"dumbfuck" + 0.000*"wilber" + 0.000*"griffith" + 0.000*"inquisitive" + 0.000*"illegible_text" + 0.000*"tequila"
INFO:gensim.models.ldamodel:topic #5 (0.844): 0.008*"patient" + 0.008*"know" 

INFO:gensim.models.ldamodel:topic #9 (0.006): 0.000*"glyni" + 0.000*"icebreaker" + 0.000*"beefy" + 0.000*"donald" + 0.000*"dumbfuck" + 0.000*"wilber" + 0.000*"griffith" + 0.000*"inquisitive" + 0.000*"illegible_text" + 0.000*"tequila"
INFO:gensim.models.ldamodel:topic #5 (0.871): 0.008*"know" + 0.006*"seem" + 0.006*"doctor" + 0.006*"patient" + 0.004*"learn" + 0.004*"die" + 0.004*"girl" + 0.004*"speak" + 0.004*"guard" + 0.004*"write"
INFO:gensim.models.ldamodel:topic #4 (2.490): 0.022*"door" + 0.011*"look" + 0.011*"walk" + 0.010*"sit" + 0.010*"bed" + 0.009*"know" + 0.008*"sleep" + 0.008*"hear" + 0.008*"window" + 0.008*"remember"
INFO:gensim.models.ldamodel:topic #2 (2.636): 0.017*"eye" + 0.009*"look" + 0.008*"scream" + 0.006*"voice" + 0.006*"arm" + 0.006*"pull" + 0.005*"stand" + 0.005*"know" + 0.005*"sound" + 0.005*"blood"
INFO:gensim.models.ldamodel:topic diff=0.024501, rho=0.196116
INFO:gensim.models.ldamodel:PROGRESS: pass 9, at document #1300/1600
INFO:gensim.models.ldamodel:optimize

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


In [175]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

KeyboardInterrupt: 

### Coherence

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

## Run Pipe

In [139]:
# def remove_markdown(doc):
    
#     text = BeautifulSoup(' '.join(doc)).text
#     return nlp.make_doc(text)

# def remove_special_chars(doc):
#     text = BeautifulSoup(' '.join(doc)).text
#     return nlp.make_doc(text)

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def lowercase(doc):
#     doc = [token.lower_ for token in doc]
#     doc = ' '.join(doc)
#     return nlp.make_doc(doc)

# def lemmatizer(doc):
#     # This takes in a doc of tokens from the NER and lemmatizes them. 
#     # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
#     doc = [token.lemma_ for token in doc]
#     doc = ' '.join(doc)
#     return nlp.make_doc(doc)

# def filter_doc(doc):
#     doc = [token.text for token in doc 
#            if token.is_space != True
#            and token.like_url != True 
#            and token.like_email != True 
#            and token.is_punct != True 
#           ] 
#     doc = ' '.join(doc)
#     return nlp.make_doc(doc)

# def norm(doc):
#     doc = [token.norm_ for token in doc] 
#     doc = ' '.join(doc)
#     return nlp.make_doc(doc)

# def remove_pronouns(doc):
#     # This will remove stopwords and punctuation and whitespace.
#     # Use token.text to return strings, which we'll need for Gensim.
#     doc = [token.text for token in doc if token.lemma_ != '-PRON-'] 
#     doc = ' '.join(doc)
#     return nlp.make_doc(doc)

# def keep_pos(doc):
#     # This will remove stopwords and punctuation and whitespace.
#     # Use token.text to return strings, which we'll need for Gensim.
#     doc = [token.text for token in doc 
#            if token.pos_ == 'NOUN' 
#            or token.pos_ == 'VERB'] 
#     doc = ' '.join(doc)
#     return nlp.make_doc(doc)

# def remove_stopwords(doc):
#     # This will remove stopwords and punctuation and whitespace.
#     # Use token.text to return strings, which we'll need for Gensim.
#     doc = [token.text for token in doc if not token.is_stop] 
#     doc = ' '.join(doc)
#     return nlp.make_doc(doc)

# def x_remove_stopwords(texts):
#     return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


In [ ]:
# Remove Stop Words
#data_words_nostops = x_remove_stopwords(data_words)
#STOP_WORDS == nlp.Defaults.stop_words

In [140]:
# nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])

# # Updates spaCy's default stop words list with my additional words. 
# nlp.Defaults.stop_words |= stop_words
# # Iterates over the words in the stop words list and resets the "is_stop" flag.
# for word in STOP_WORDS:
#     lexeme = nlp.vocab[word]
#     lexeme.is_stop = True
    

# #merge_ents = nlp.create_pipe("merge_entities")
# #nlp.add_pipe(merge_ents,after='lemmatizer')    
# #merge_nps = nlp.create_pipe("merge_noun_chunks")
# #nlp.add_pipe(merge_nps, after='parser')
# # s2v = Sense2VecComponent(nlp.vocab).from_disk("/Users/kirill.kogan/Documents/data/models/sense2vec-vectors/")
# # nlp.add_pipe(s2v)    
    
# #nlp.add_pipe(remove_markdown, name='remove_markdown', first=True)
# #nlp.add_pipe(lemmatizer, name='lemmatizer', before='ner')

# #nlp.add_pipe(keep_pos, name='keep_pos')
# nlp.add_pipe(filter_doc, name='filter_doc')
# nlp.add_pipe(lowercase, name='lowercase')
# nlp.add_pipe(norm, name="norm")
# nlp.add_pipe(remove_stopwords, name="remove_stopwords")



//anaconda3/envs/reddit/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
//anaconda3/envs/reddit/lib/python3.7/site-packages/msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)


In [116]:
# print(nlp.pipe_names)

['tagger', 'filter_doc', 'lowercase', 'norm', 'remove_stopwords']


In [65]:
# bigram = gensim.models.Phrases(docs_a)
# bigrammed = [bigram[doc] for doc in docs_a]


INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:collected 5632 word types from a corpus of 8084561 words (unigram + bigrams) and 800 sentences
INFO:gensim.models.phrases:using 5632 counts as vocab in Phrases<0 vocab, min_count=5, threshold=10.0, max_vocab_size=40000000>


### sample pipe

1

In [141]:
# doc = nlp(docs_a[random.randint(0, len(docs_a))])
# print([token.text for token in doc if not token.is_stop])

NameError: name 'cos_a' is not defined

### Run Pipe

In [142]:
# doc_list = []
# # Iterates through each article in the corpus.
# for doc in  nlp.pipe(tqdm_notebook(docs_a), batch_size=10, n_threads=WORKERS):
#     #text = [token.text for token in doc]
#     doc_list.append([token.text for token in doc])


//anaconda3/envs/reddit/lib/python3.7/site-packages/tqdm/__init__.py:27: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  TqdmDeprecationWarning)


### Phrases

In [143]:
# # Form Bigrams
# data_words_bigrams = make_bigrams(doc_list)

In [144]:
# data_words_bigrams[0:5]

[['kid',
  'divorce',
  'arms',
  'legs',
  'abusing',
  'drugs',
  'heavily',
  'earth',
  'fine',
  'lived',
  'dad',
  'anyways',
  'abusive',
  'fuck',
  'dating',
  'smile',
  'sweating',
  'chested',
  'workout',
  'lay',
  'couch',
  'pillow',
  'pass',
  'beat',
  'mistook',
  'bb',
  'gun',
  'gun',
  'gun',
  'dad',
  'conversation',
  'baseball_bat',
  'stayed',
  'dad',
  'permanently',
  'bills',
  'kept',
  'harder',
  'harder',
  'eventually',
  'uncle',
  'donnie',
  'donnie',
  'interesting',
  'walk',
  'tighty',
  'whities',
  'honesty',
  'tighty',
  'yellows',
  'mail',
  'mow',
  'lawn',
  'answer',
  'doors',
  'sons',
  'bitches',
  'underwear',
  'stood',
  'profile',
  'belly',
  'massive',
  'bloated',
  'melted',
  'waistline',
  'muffin',
  'muffin',
  'cup',
  'needless',
  'individual',
  'dad',
  'stressed',
  'fused',
  'angry',
  'yelling',
  'lungs',
  'cursing',
  'hear',
  'directions',
  'remember',
  'sitting',
  'rants',
  'turning',
  'purple',


In [ ]:
# Turns each document into a bag of words.
#words = corpora.Dictionary(doc_list)
#corpus = [words.doc2bow(doc) for doc in doc_list]
#pprint.pprint(words)

# Topics

### Overview

In [ ]:
# make binary count vectorizer X NMF
# make count vectorizer X NMF
# make tfidf vectorizer X NMF

# make binary count vectorizer X LDA
# make count vectorizer X LDA
# make tfidf vectorizer X LDA

# make binary count vectorizer X COREX
# make count vectorizer X COREX
# make tfidf vectorizer X COREX


# look into BERT
# 

## Topic Text Params